In [1]:
import json
import requests
from config import fed_api_key
from pprint import pprint
import pandas as pd
import numpy as np
import datetime

In [2]:
commodities_dict= {'lumber':'WPU081','cement':'PCU32733273','steel':'WPU101','stone':'WPS1321','brick':'PCU32733132733106'}

In [3]:
#'https://api.stlouisfed.org/fred/series/observations?series_id=WPU0811&api_key=0a60bfbafed5ab7080d559305d53573a&file_type=json'
base_url='https://api.stlouisfed.org/fred/series/observations?series_id='
mid_url = '&api_key='
end_url='&file_type=json'
urls =[]
dummies = []

In [4]:
commodity_name=[]
dates = []
values = []

for key,value in commodities_dict.items():
    url = base_url + value + mid_url + fed_api_key + end_url
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
         dates.append(data['date'])
         values.append(data['value'])
         commodity_name.append(key)
df = pd.DataFrame({
     "Commodity": commodity_name,
     "date": dates,
     "values": values
})
df['date']=pd.to_datetime(df['date'],infer_datetime_format=True)
df.to_csv('Resources/ConstructionCommodities.csv')
df.head(10)

,Commodity,date,values
0,lumber,1926-01-01,8.3
1,lumber,1926-02-01,8.3
2,lumber,1926-03-01,8.3
3,lumber,1926-04-01,8.2
4,lumber,1926-05-01,8.1
5,lumber,1926-06-01,8.0
6,lumber,1926-07-01,8.0
7,lumber,1926-08-01,7.9
8,lumber,1926-09-01,7.9
9,lumber,1926-10-01,7.9


In [6]:
file = "Resources/ZilloHomeValueIndexZip.csv"
zhvi_df = pd.read_csv(file)
zhvi_df = zhvi_df[zhvi_df['State']=='TX']
zhvi_df['Country'] = 'US'
zhvi_df.head(10)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,Country
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,225325.0,...,354939.0,358995.0,363611.0,369082.0,375735.0,382697.0,391365.0,402167.0,413199.0,US
5,91940,5,77449,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,107110.0,...,206612.0,208843.0,211725.0,213130.0,215301.0,217929.0,223432.0,229436.0,235859.0,US
7,91733,7,77084,Zip,TX,TX,Houston,Houston-The Woodlands-Sugar Land,Harris County,106353.0,...,204784.0,206993.0,209809.0,211363.0,213561.0,215995.0,220991.0,226551.0,232848.0,US
8,93144,8,79936,Zip,TX,TX,El Paso,El Paso,El Paso County,90628.0,...,147287.0,148997.0,150717.0,152543.0,154337.0,156983.0,159868.0,162910.0,165737.0,US
12,92593,12,78660,Zip,TX,TX,Pflugerville,Austin-Round Rock,Travis County,164957.0,...,313724.0,319220.0,328431.0,341024.0,359850.0,378330.0,400738.0,419873.0,435000.0,US
22,92271,23,78130,Zip,TX,TX,New Braunfels,San Antonio-New Braunfels,Comal County,NaN,...,259301.0,261377.0,264493.0,267235.0,271195.0,276050.0,283636.0,291992.0,300023.0,US
26,92036,27,77573,Zip,TX,TX,League City,Houston-The Woodlands-Sugar Land,Galveston County,NaN,...,293207.0,296235.0,299196.0,302832.0,306433.0,312854.0,319781.0,327324.0,332734.0,US
28,92045,29,77584,Zip,TX,TX,Pearland,Houston-The Woodlands-Sugar Land,Brazoria County,172470.0,...,281784.0,284030.0,286128.0,288596.0,292311.0,295883.0,301769.0,308621.0,316827.0,US
33,92551,34,78613,Zip,TX,TX,Cedar Park,Austin-Round Rock,Williamson County,181612.0,...,371938.0,377377.0,389197.0,407058.0,433251.0,457938.0,484085.0,505646.0,524660.0,US
34,91926,35,77433,Zip,TX,TX,Cypress,Houston-The Woodlands-Sugar Land,Harris County,179240.0,...,307196.0,310097.0,314357.0,316746.0,320292.0,324102.0,332041.0,340997.0,350337.0,US


In [20]:
zhvi_df = zhvi_df.drop(columns=['SizeRank','RegionID','RegionType','StateName'])
zhvi2_df = pd.melt(zhvi_df,id_vars=['Country','RegionName','Metro','City','State','CountyName'],var_name='Date', value_name='MedianPrice')
zhvi2_df = zhvi2_df.rename(columns={"State": "UsState"})
zhvi2_df = zhvi2_df.rename(columns={"RegionName": "Zipcode"})
zhvi2_df['Date']=pd.to_datetime(zhvi2_df['Date'],infer_datetime_format=True)
zhvi2_df['MeasureDate'] = zhvi2_df['Date'] + pd.DateOffset(1)
zhvi2_df = zhvi2_df.drop(columns=['Date'])
zhvi2_df

,Country,Zipcode,Metro,City,UsState,CountyName,MedianPrice,MeasureDate
0,US,77494,Houston-The Woodlands-Sugar Land,Katy,TX,Harris County,225325.0,2000-02-01
1,US,77449,Houston-The Woodlands-Sugar Land,Katy,TX,Harris County,107110.0,2000-02-01
2,US,77084,Houston-The Woodlands-Sugar Land,Houston,TX,Harris County,106353.0,2000-02-01
3,US,79936,El Paso,El Paso,TX,El Paso County,90628.0,2000-02-01
4,US,78660,Austin-Round Rock,Pflugerville,TX,Travis County,164957.0,2000-02-01
...,...,...,...,...,...,...,...,...
466175,US,76467,Dallas-Fort Worth-Arlington,Paluxy,TX,Hood County,208768.0,2021-09-01
466176,US,79326,NaN,Fieldton,TX,Lamb County,135622.0,2021-09-01
466177,US,77428,Bay City,Palacios,TX,Matagorda County,125869.0,2021-09-01
466178,US,3575,NaN,Clifton,TX,Bosque County,569863.0,2021-09-01


In [21]:
zhvi2_df.to_csv('Resources/zhvi_zip.csv')